In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import math
import warnings

from IPython.display import display
from sklearn import linear_model
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [2]:
df = pd.read_csv('C:\\Users\\STEVEN\\Documents\\Thinkful\\florida_crime.csv')

In [3]:
df = df.fillna(0)
print(df.loc[0,:])

City                                      Alachua
Population                                  9,338
Violent\ncrime                                 36
Murder and\nnonnegligent\nmanslaughter      0.000
Rape\n(revised\ndefinition)1                2.000
Rape\n(legacy\ndefinition)2                 0.000
Robbery                                         2
Aggravated\nassault                            32
Property\ncrime                               166
Burglary                                       40
Larceny-\ntheft                               120
Motor\nvehicle\ntheft                           6
Arson                                       1.000
Unnamed: 13                                 0.000
Name: 0, dtype: object


In [4]:
#lets create some features and clean up

df['Population'] = df['Population'].str.replace(',', '')
df = df.fillna(0)
df['Population'] = df['Population'].astype(int)

In [5]:
df['Property\ncrime'] = df['Property\ncrime'].str.replace(',', '')
df = df.fillna(0)
df['Property\ncrime'] = df['Property\ncrime'].astype(int)

df['Violent\ncrime'] = df['Violent\ncrime'].str.replace(',', '')
df = df.fillna(0)
df['Violent\ncrime'] = df['Violent\ncrime'].astype(int)

df['Larceny-\ntheft'] = df['Larceny-\ntheft'].str.replace(',', '')
df = df.fillna(0)
df['Larceny-\ntheft'] = df['Larceny-\ntheft'].astype(int)

In [6]:

df['Aggravated\nassault'] = df['Aggravated\nassault'].str.replace(',', '')
df = df.fillna(0)
df['Aggravated\nassault'] = df['Aggravated\nassault'].astype(int)



In [7]:
#add categorical aggravated assault 
df['assualt_cat'] = df['Aggravated\nassault'].apply(lambda x: 0 if x == 0 else 1)

In [8]:
df['Motor\nvehicle\ntheft'] = df['Motor\nvehicle\ntheft'].str.replace(',', '')
df = df.fillna(0)
df['Motor\nvehicle\ntheft'] = df['Motor\nvehicle\ntheft'].astype(int)

In [9]:
#Create population squared
df['Population_sq'] = df['Population'].apply(lambda x: x*x)
# Robbery
df['Robbery'] = df['Robbery'].str.replace(',', '')
df = df.fillna(0)
df['Robbery'] = df['Robbery'].astype(int)
#Creating categorical murders
df = df.rename(index=str,columns={'Murder and\nnonnegligent\nmanslaughter': 'Murder'})
df['Murder'] = df['Murder'].astype(int)
df['murder_cat'] = df['Murder'].apply(lambda x: 0 if x == 0 else 1)

In [10]:
df['Population_sqrt'] = np.sqrt(df['Population'])

df['Burglary'] = df['Burglary'].str.replace(',', '')
df = df.fillna(0)
df['Burglary'] = df['Burglary'].astype(int)
df['burglary_cat'] = df['Burglary'].apply(lambda x: 0 if x == 0 else 1)

In [24]:
print(X.shape)

(293, 16)


In [11]:
df.shape

(293, 19)

In [12]:
X = df.drop(['City', 'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2'], axis=1)
Y = df['Population']

In [13]:
#Create training and test data
trainsize = int(df.shape[0] / 2)
df_test = X.iloc[trainsize:, :].copy()
df_train = X.iloc[:trainsize, :].copy()

#Train data
Y_train = df_train['Population'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['Population'])]

#Test data
Y_test = df_test['Population'].values.reshape(-1, 1)
X_test = df_test.loc[:, ~(df_train.columns).isin(['Population'])]

In [14]:
#Logistic Regression training data:


from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=10)

# Fit the model
fit = lr.fit(X_train, Y_train)
fit2 = lr.fit(X_train, Y_train)
# Display.



print('\n R2')
print(lr.score(X_train, Y_train))


 R2
0.00684931506849315


c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
#Now to run it with our holdout comparison and cross folding.  
from sklearn.model_selection import train_test_split

X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(lr.fit(X_train2, Y_train2).score(X_test2, Y_test2)))
print('Testing on Sample: ' + str(lr.fit(X, Y).score(X, Y)))


With 20% Holdout: 0.0
Testing on Sample: 0.006825938566552901


In [25]:
#Now run as cross validation: 
from sklearn.model_selection import cross_val_score
cross_val_score(lr, X_test, Y_test, cv=2)


c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0.02068966, 0.        ])

In [17]:
# now it's time for ridge regression..
from sklearn import linear_model
from sklearn import preprocessing

ridgeregr = linear_model.Ridge(alpha=5) 

# Fit the model
fit = ridgeregr.fit(X_train, Y_train)

# Display.



print('\n R2')
print(ridgeregr.score(X_train, Y_train))



 R2
0.9935522467129367


c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.785425e-24
  overwrite_a=True).T


In [18]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(ridgeregr.fit(X_train2, Y_train2).score(X_test2, Y_test2)))
print('Testing on Sample: ' + str(ridgeregr.fit(X, Y).score(X, Y)))

With 20% Holdout: 1.0
Testing on Sample: 1.0


c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.411821e-23
  overwrite_a=True).T
c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.836555e-24
  overwrite_a=True).T


In [26]:
from sklearn.model_selection import cross_val_score
cross_val_score(ridgeregr, X_test, Y_test, cv=10)

c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.994041e-22
  overwrite_a=True).T
c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.239127e-23
  overwrite_a=True).T
c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.204468e-23
  overwrite_a=True).T
c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditio

array([0.98659773, 0.97754034, 0.98427406, 0.96249637, 0.95856894,
       0.93148938, 0.99417427, 0.9854224 , 0.97728023, 0.7794973 ])

In [20]:
#let's do lasso next

In [21]:
lass = linear_model.Lasso(alpha=1000)

# Fit the model
fit = lass.fit(X_train, Y_train)

# Display.



print('\n R2')
print(lass.score(X_train, Y_train))



 R2
0.9925455766231105


c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [22]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(lass.fit(X_train2, Y_train2).score(X_test2, Y_test2)))
print('Testing on Sample: ' + str(lass.fit(X, Y).score(X, Y)))

With 20% Holdout: 0.9999999999990283
Testing on Sample: 0.9999999999999657


In [27]:
from sklearn.model_selection import cross_val_score
cross_val_score(lass, X_test, Y_test, cv=10)

c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\steven\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\steven\appdata\local\programs\python\python36-32\lib\si

array([0.98202908, 0.97722854, 0.97710625, 0.95498133, 0.96534506,
       0.93112072, 0.99425311, 0.9895635 , 0.97538408, 0.68084883])

There were a few different interesting things that were discovered during the analysis.  

While searching through the data we included a few different features in predicting population with crime.  We had population squared and the square root of population in there, to see if they were functional.  We included for low frequency crimes a categorical feature for them.  For the features of murder, burglary, and assault.

After looking through the different models.  And testing on test and training sets.  It looks like the ridge regression model is the most efficient amongst the test and training methods.  Although it does appear that the model may be overfitting.  Second I would choose lasso because it helps prevent overfitting and may prevent a more honest representation.  